# Q2: Simulating Attitude Dynamics (50 pts)
We're going to simulate the attitude dynamics of a rigid body like this T-wrench: https://www.youtube.com/watch?v=1n-HMSCDYtM

In [12]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/devel/dynamics-simulation-16-715/HW1_solutions/Project.toml`
Precompiling project...
  ✓ Tables
  ✓ StructArrays
  ✓ SciMLBase
  ✓ DEDataArrays
  ✓ GeometryBasics
  ✓ NonlinearSolve
  ✓ DiffEqBase
  ✓ Plots
  ✓ OrdinaryDiffEq
  9 dependencies successfully precompiled in 128 seconds (177 already precompiled, 1 skipped during auto due to previous errors)
  9 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [81]:
using LinearAlgebra
using ForwardDiff
using OrdinaryDiffEq
using Test
using Random
using Plots
plotly()

Plots.PlotlyBackend()

In [3]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes
function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I + hat(q[2:4])]
end

function R(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I - hat(q[2:4])]
end

H = [zeros(1,3); I]

4×3 Matrix{Float64}:
 0.0  0.0  0.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [4]:
#Inertia matrix we'll be using
J = Diagonal([1.0; 2.0; 3.0])

3×3 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅ 
  ⋅   2.0   ⋅ 
  ⋅    ⋅   3.0

In [89]:
#Implement Euler's equation for a rigid body
function dynamics_ω(ω)
    # TODO: add Euler's equation 
    ω̇ = zeros(length(ω))
    
    # solution
    ω̇ = -J\(hat(ω)*J*ω)
    
    return ω̇
end

@testset "Euler's" begin
    ω1 = [.2;7;.2]
    α1 = [-1.4;0.04;-0.46666666666666]
    @test isapprox(α1, dynamics_ω(ω1),atol=1e-12)
end

Test Summary: | Pass  Total
Euler's       |    1      1


Test.DefaultTestSet("Euler's", Any[], 1, false, false)

In [88]:
#Implement the full attitude dynamics using a rotation matrix
function dynamics_Q(x)
    # input: x = [vec(Q);ω]
    
    # TODO: implement 
    ẋ = zeros(length(x))
    
    # solution
    Q = reshape(x[1:9],3,3)
    ω = x[10:12]
    Q̇ = Q*hat(ω)
    ẋ = [Q̇[:]; dynamics_ω(ω)]
    
    return ẋ
end

function dynamics_Q!(ẋ,x,p,t)
    # make the ODE in-place so it works with OrdinaryDiffEq
    ẋ .= dynamics_Q(x)
end

@testset "attitude dynamics (DCM)" begin 
    Q1 = exp(hat([1;2;3]))
    ω1 = [0.2;0.3;0.7]
    x1 = [vec(Q1);ω1]
    ẋ1 = [0.47267683571087754,-0.4926072371847232,0.337512546219523,
          0.5043029621213005,0.3210433517191289, -0.41546322185318607,
         -0.35118036539795094,0.003154917030294274,0.08162351044578747,
         -0.2099999999999999,0.14,-0.02]
    
    @test isapprox(ẋ1,dynamics_Q(x1),atol=1e-12)
end

Test Summary:           | Pass  Total
attitude dynamics (DCM) |    1      1


Test.DefaultTestSet("attitude dynamics (DCM)", Any[], 1, false, false)

In [94]:
#Implement the full attitude dynamics using a quaternion

function dynamics_q(x)
    
    # TODO: implement attitude dynamics with quaternion 
    ẋ = zeros(length(x))
    
    # solution 
    q = x[1:4]
    ω = x[5:7]
    q̇ = 0.5*L(q)*H*ω
    ẋ = [q̇; dynamics_ω(ω)]
    
    return ẋ
end

function dynamics_q!(ẋ,x,p,t)
    # make the ODE in-place so it works with OrdinaryDiffEq
    ẋ .= dynamics_q(x)
end

@testset "attitude dynamics (quaternion)" begin
    r1 = normalize([1;2;3])
    θ1 = deg2rad(45)
    q1 = [cos(θ1/2);r1*sin(θ1/2)]
    x1 = [q1;0.2;7;0.3]
    ẋ1 = [-0.7721871929186838;
         -0.9508318305595436;
          3.2489198311984837;
          0.47609421287426346;
         -2.0999999999999996;
          0.06000000000000001;
         -0.46666666666666673]
    @test isapprox(dynamics_q(x1),ẋ1,atol=1e-12)
    
end

Test Summary:                  | Pass  Total
attitude dynamics (quaternion) |    1      1


Test.DefaultTestSet("attitude dynamics (quaternion)", Any[], 1, false, false)

In [8]:
#Initial Conditions
Q0 = Array(I(3))
q0 = [1.0; 0; 0; 0]
Random.seed!(1234)
ω0 = [0; 2*pi; 0] + 1e-2*randn(3) #small perturbation from intermediate-axis spin

x0Q = [Q0[:]; ω0]
x0q = [q0; ω0]

7-element Vector{Float64}:
  1.0
  0.0
  0.0
  0.0
 -0.024767621973547548
  6.29801711113256
 -0.00451048881824724

In [9]:
#Set up and solve with DifferentialEquations.jl with default settings
Tf = 60.0
tspan = (0.0, Tf)

prob_q = ODEProblem(dynamics_q!, x0q, tspan)
prob_Q = ODEProblem(dynamics_Q!, x0Q, tspan)

sol_q = solve(prob_q,Tsit5());
sol_Q = solve(prob_Q,Tsit5());

In [10]:
#ToDo: Make a meshcat animation of the T-wrench here
plot(sol_q, vars=(5,6,7))

In [23]:
using MeshCat
using CoordinateTransformations
using Rotations
using GeometryBasics: HyperRectangle, Vec, Point, Mesh
using Colors: RGBA, RGB

# Create a new visualizer instance
vis = Visualizer()
render(vis)


┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /Users/kevintracy/.julia/packages/MeshCat/RLYij/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8703))

In [69]:
# meshcat stuff
# delete!(vis)
t_vec = 1:0.01:60
q_hist = sol_q(t_vec).u
box = HyperRectangle(Vec(-0.1, -1, -0.1), Vec(0.2, 2, 0.2))
box2 = HyperRectangle(Vec(-0.1 - 1, -1, -0.1), Vec(2, 0.2, 0.2))
green_material = MeshPhongMaterial(color=RGBA(1, 1, 0.3, 0.5))
setobject!(vis["group1"]["greenbox"], box, green_material)
blue_material = MeshPhongMaterial(color=RGBA(0, 0, 1, 0.5))
setobject!(vis["group1"]["bluebox"], box2, blue_material)
settransform!(vis["group1"], Translation(0, 0, 0))
# animation loop
for i = 1:length(t_vec)
    settransform!(vis, LinearMap(UnitQuaternion(q_hist[i][1:4])  ))
    sleep(0.005)
end

In [70]:
#Plot the error in the quaternion norm vs. time
qnorm = zeros(length(sol_q.t))
for k = 1:length(sol_q.t)
    qnorm[k] = norm(sol_q.u[k][1:4])
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="",lw=2)

In [71]:
#Plot the orthogonality error vs. time for the rotation matrix
Qerr = zeros(length(sol_Q.t))
for k = 1:length(sol_Q.t)
    Qk = reshape(sol_Q.u[k][1:9],3,3)
    Qerr[k] = tr(Qk'*Qk - I(3))
end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="",lw=2)

If you're writing your own integrator, you can easily re-normalize the quaternion at each step. Another option that can work if you're using an off-the-shelf ODE solver is to add proportional feedback on the quaternion norm error to the dynamics. This is an example of an idea called Baumgarte stabilization that can be used to make Runge-Kutta methods better respect known constraints and conserved quantities. Implement it and test some feedback gains between 1 and 20 before choosing a value that you find works best.

In [72]:
function dynamics_q_stabilized(x)
    q = x[1:4]
    ω = x[5:7]
    
    Kp = 12.0
    
    q̇ = 0.5*L(q)*H*ω - Kp*(norm(q)-1).*q
    ω̇ = -J\(hat(ω)*J*ω)
    
    ẋ = [q̇; ω̇]
end

function dynamics_q_stabilized!(ẋ,x,p,t)
    ẋ .= dynamics_q_stabilized(x)
end

dynamics_q_stabilized! (generic function with 1 method)

In [73]:
prob_q2 = ODEProblem(dynamics_q_stabilized!, x0q, tspan)
sol_q2 = solve(prob_q2,Tsit5());

In [74]:
#Plot norm(q) with and without stabilization
qnorm2 = zeros(length(sol_q2.t))
for k = 1:length(sol_q2.t)
    qnorm2[k] = norm(sol_q2.u[k][1:4])
end

plot(sol_q.t,qnorm,xlabel="Time (sec)",ylabel="||q||",label="No Stabilization",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)

Unfortunately, the stabilization idea doesn't work as well with rotation matrices since the constraints are much more complicated. Instead, we can implement a Runge-Kutta method that automatically preserves orthogonality of rotation matrices (or normalization of quaternions) by using the exponential map and group multiplication operations. Let's do this for the rotation matrix case first by modifying the explicit midpoint method. This is a simple example of what's known as a Runge-Kutta-Munthe-Kaas method. It is possible to construct an RK-MK method that stays on a Lie group from any standard RK method.

In [75]:
function lie_midpoint_step_Q(xk,h)
    
    Qk = reshape(xk[1:9],3,3)
    ωk = xk[10:12]
    
    ω̇k = dynamics_ω(ωk)
    ωm = ωk+0.5*h*ω̇k
    ω̇m = dynamics_ω(ωm)
    
    ωn = ωk + h*ω̇m
    
    ΔQ = exp(hat(h*ωm))
    
    Qn = Qk*ΔQ
    
    return [Qn[:]; ωn]
end

lie_midpoint_step_Q (generic function with 1 method)

In [76]:
#Now let's simulate the same example and plot the orthogonality error again

h = 0.1
N = N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1))

xtrajQ = zeros(12,N)
xtrajQ[:,1] .= x0Q

for k = 1:(N-1)
    xtrajQ[:,k+1] .= lie_midpoint_step_Q(xtrajQ[:,k], h)
end

Qerr2 = zeros(N)
for k = 1:N
    Qk = reshape(xtrajQ[1:9,k],3,3)
    Qerr2[k] = tr(Qk'*Qk - I(3))
end

plot(sol_Q.t,Qerr,xlabel="Time (sec)",ylabel="Orthogonality Error",label="RK",lw=2)
plot!(thist,Qerr2,label="Lie Group",lw=2)

Now let's implement the same Lie group idea but use quaternions instead of rotation matrices. To do this, we'll also need to implement the quaternion exponential map. See if you can figure out a way to implement it without introducing divide-by-zero issues when the angular velocity goes to zero (hint: look into the sinc function).

In [77]:
function Expq(ϕ)
    θ = norm(ϕ)
    q = [cos(θ/2); 0.5*ϕ*sinc(θ/(2*pi))]
end

function lie_midpoint_step_q(xk,h)
    
    qk = xk[1:4]
    ωk = xk[5:7]
    
    ω̇k = dynamics_ω(ωk)
    ωm = ωk+0.5*h*ω̇k
    ω̇m = dynamics_ω(ωm)
    
    ωn = ωk + h*ω̇m
    
    Δq = Expq(h*ωm)
    
    qn = L(qk)*Δq
    
    return [qn; ωn]
end

lie_midpoint_step_q (generic function with 1 method)

In [78]:
#Now let's simulate and plot the quaternion norm again

xtrajq = zeros(7,N)
xtrajq[:,1] .= x0q

for k = 1:(N-1)
    xtrajq[:,k+1] .= lie_midpoint_step_q(xtrajq[:,k], h)
end

qnorm3 = zeros(N)
for k = 1:N
    qnorm3[k] = norm(xtrajq[1:4,k])
end

plot(sol_q.t,qnorm,label="No Stabilization",xlabel="Time (sec)",ylabel="||q||",lw=2)
plot!(sol_q2.t,qnorm2,label="Stabilization",lw=2)
plot!(thist,qnorm3,label="Lie Group",lw=2)